### 梯度：将标量的导数 拓展到 向量

In [ ]:
# 梯度指向的是函数变化最大的方向，与等高线正交

In [ ]:
# pytorch采用的是自动求导和计算图，所以不需要进行手动求导
# 但 导数是基础，需要理解导数的形状和input的形状是什么样的关系

### 自动求导：计算一个函数在指定值上的导数

概念：计算图</br>
类似于链式求导法则，计算机将函数一步一步分解生成有向图，然后根据图进行计算；</br>
显式构造：tensorflow（基于数学公式 构造计算图）</br>
隐式构造：pytorch（程序帮忙记录下来整个数学计算过程）</br>

方式：基于链式法则 进行 正向累积 或者 反向累积（又称为反向传递）

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
x.requires_grad_(True) # 告诉pytorch需要存储梯度，存储于x.grad
x.grad

In [4]:
y = 2 * torch.dot(x, x)
y # 返回的结果中有grad_fn

tensor(28., grad_fn=<MulBackward0>)

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [6]:
x.grad == 4*x

tensor([True, True, True, True])

In [9]:
# 默认情况下，pytorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward() 
x.grad

tensor([1., 1., 1., 1.])

In [10]:
x.grad.zero_()
y = x*x
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

### 将某些计算移动到记录的计算图之外

In [11]:
x.grad.zero_()
y = x*x
u = y.detach()
z = u*x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [12]:
x.grad.zero_()
y.sum().backward()
x.grad == 2*x

tensor([True, True, True, True])

### 即使构造函数的计算图需要通过python控制流（如 条件、循环），我们仍然可以计算得到变量的梯度

此处表明 pytorch中隐式求导比显式求导 在控制流上做的好一点；
但反过来说，pytorch此处也会慢一点；

## QA

1. Q:为什么获取grad前需要backward?</br>
A: 因为计算梯度是很耗内存和时间的一件事，除非显式指定，否则不会自动计算梯度；